In [ ]:
'''
LLM Take-home assignment round
Programma'cally solve for these ques'ons. Try to answer as many ques'ons as possible. Try
to complete in 48 hours. Some of these may require explora'on, so do try.
1. Use a pre-trained google/flan-t5-small as the model.
2. Verify if the summariza'on task works.
3. Verify if the Q&A task works.
4. Verify if English to French transla'on task works.
5. Programma'cally print the names of all the model layers and their dimensions.
6. Programma'cally print the total number of parameters/weights in this model.
7. Set the tensor in final layer (decoder.final_layer_norm.weight) to all zeros.
8. Verify if the Q&A task works aWer reseXng the weights of the above layer.
9. Replace the decoder.final_layer_norm.weight with a layer of smaller dimensions and
adjust all the dependent layers to match the dimension
10. Reload the original google/flan-t5-small model.
11. Train the model for a Q&A task that takes a context as addi'onal input along with the
ques'on. You can use SQuAD dataset (h_ps://rajpurkar.github.io/SQuAD-explorer/ )
or the smaller Topioca dataset (h_ps://mcgill-nlp.github.io/topiocqa/) . Choose an
appropriate task prefix/trigger word and jus'fy the choice.
12. Evaluate the quality of the model
Next discussion will be around the solu'on and geXng deeper into certain
algorithms
'''


# Dependencies installation

In [ ]:
! pip install transformers sentencepiece accelerate evaluate rouge_score
! pip install evaluate

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [46]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

In [4]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie ich er bitten?</s>


# Summarization

In [22]:
# CNN dailymail example
summary_cnn_dailymail_input = {
  'id': "0054d6d30dbcad772e20b22771153a2a9cbeaf62",
  'article': "(CNN) -- An American woman died aboard a cruise ship that docked at Rio de Janeiro on Tuesday, the same ship on which 86 passengers previously fell ill, according to the state-run Brazilian news agency, Agencia Brasil. The American tourist died aboard the MS Veendam, owned by cruise operator Holland America. Federal Police told Agencia Brasil that forensic doctors were investigating her death. The ship's doctors told police that the woman was elderly and suffered from diabetes and hypertension, according the agency. The other passengers came down with diarrhea prior to her death during an earlier part of the trip, the ship's doctors said. The Veendam left New York 36 days ago for a South America tour.",
  'highlights': "The elderly woman suffered from diabetes and hypertension, ship's doctors say .\nPreviously, 86 passengers had fallen ill on the ship, Agencia Brasil says ."}


In [23]:

summary_input_text = "Summarize the highlights of this article.: " + summary_cnn_dailymail_input['article']
summary_input_tokens = tokenizer(summary_input_text, return_tensors="pt").input_ids.to("cuda")
summary_model_op_tokens = model.generate(summary_input_tokens)
summarisation_ouput = tokenizer.decode(summary_model_op_tokens[0], skip_special_tokens=True)
print(summarisation_ouput)

NEW: Agencia Brasil says 86 passengers died aboard the same ship. NEW


# Q&A

In [30]:
qna_input = {
    "question" :"Answer the following yes/no question. Can you write a whole Haiku in a single tweet?"
    }

In [33]:
qna_input_tokens = tokenizer(qna_input['question'], return_tensors="pt").input_ids.to("cuda")
qna_op_tokens = model.generate(qna_input_tokens)
print(tokenizer.decode(qna_op_tokens[0],skip_special_tokens=True))


No


# Translation

In [38]:
translation_input = {"english" : "I have a reservation", "French" : "J'ai une réservation."}

In [39]:
translation_input['english']

'I have a reservation'

In [41]:
translation_input_tokens =tokenizer("translate english to french " + translation_input['english'], return_tensors="pt").input_ids.to("cuda")
translation_op_tokens = model.generate(translation_input_tokens)
print(tokenizer.decode(translation_op_tokens[0], skip_special_tokens = True))

Vous avez un réserve.


# Print names of model layer names and dimensions

In [36]:
list(next(model.named_parameters())[1].size())

[32128, 512]

In [37]:
for name, layer in model.named_parameters():
  print("Layer name", name)
  print("Layer Dimensions", list(layer.shape))

Layer name shared.weight
Layer Dimensions [32128, 512]
Layer name encoder.block.0.layer.0.SelfAttention.q.weight
Layer Dimensions [384, 512]
Layer name encoder.block.0.layer.0.SelfAttention.k.weight
Layer Dimensions [384, 512]
Layer name encoder.block.0.layer.0.SelfAttention.v.weight
Layer Dimensions [384, 512]
Layer name encoder.block.0.layer.0.SelfAttention.o.weight
Layer Dimensions [512, 384]
Layer name encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
Layer Dimensions [32, 6]
Layer name encoder.block.0.layer.0.layer_norm.weight
Layer Dimensions [512]
Layer name encoder.block.0.layer.1.DenseReluDense.wi_0.weight
Layer Dimensions [1024, 512]
Layer name encoder.block.0.layer.1.DenseReluDense.wi_1.weight
Layer Dimensions [1024, 512]
Layer name encoder.block.0.layer.1.DenseReluDense.wo.weight
Layer Dimensions [512, 1024]
Layer name encoder.block.0.layer.1.layer_norm.weight
Layer Dimensions [512]
Layer name encoder.block.1.layer.0.SelfAttention.q.weight
Layer Dimension

# Total number of parameters in the model

In [24]:
import torch

parameter_count =0
for i in model.parameters():
  parameter_count = torch.numel(i.data)  + parameter_count

print('Total number of parameters ', parameter_count)

Total number of parameters  76961152


# Set the tensor in final layer (decoder.final_layer_norm.weight) to all zeros

In [41]:
from torch import nn

In [47]:
qna_input = {
    "question" :"What is the capital of USA?"
    }

qna_input_tokens = tokenizer(qna_input['question'], return_tensors="pt").input_ids.to('cuda')
qna_op_tokens = model.generate(qna_input_tokens)
print('Captial is ',tokenizer.decode(qna_op_tokens[0],skip_special_tokens=True),'.')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Captial is  san diego .


In [48]:
model.decoder.final_layer_norm.weight = nn.Parameter(torch.zeros_like(model.decoder.final_layer_norm.weight))

In [44]:
qna_input_tokens = tokenizer(qna_input['question'], return_tensors="pt").input_ids.to('cuda')
qna_op_tokens = model.generate(qna_input_tokens)
print('Captial is ',tokenizer.decode(qna_op_tokens[0],skip_special_tokens=True),'.')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Captial is   .


# Replace decoder.final_layer_norm.weight with smaller layer


# Config based approach

**T5LayerNorm** constructor takes a parameter hidden size which is implicitly derived from **T5Config.d_model**. So changing d_model changes the **T5LayerNorm** size

This approach needs a custom generator to pass the inputs

In [4]:
from transformers import T5Tokenizer
from transformers import T5Config
from transformers import T5Model

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
custom_T5Config = T5Config(d_model=256)

model = T5Model(custom_T5Config)

print(model.decoder.final_layer_norm.weight.shape)



torch.Size([256])


# Change the decoder.final_layer_norm

256 is passed **T5LayerNorm** constructor. Layers above dimensions also change right from first encoder block through last decoder block. This approach fails because of skip connections

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from transformers.models.t5.modeling_t5 import T5LayerNorm


tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")


for i in range(0,7):
    model.encoder.block[i].layer[0].SelfAttention.o = torch.nn.Linear(in_features=384, out_features=256, bias=False)
    model.encoder.block[i].layer[1].DenseReluDense.wo = torch.nn.Linear(in_features=1024, out_features=256, bias=False)

for i in range(0,7):
    model.decoder.block[i].layer[0].SelfAttention.o = torch.nn.Linear(in_features=384, out_features=256, bias=False)
    model.decoder.block[i].layer[1].EncDecAttention.o = torch.nn.Linear(in_features=384, out_features=256, bias=False)
    model.decoder.block[i].layer[2].DenseReluDense.wo = torch.nn.Linear(in_features=1024, out_features=256, bias=False)


model.decoder.final_layer_norm = T5LayerNorm(256)
model.lm_head = torch.nn.Linear(in_features=256, out_features=32128, bias=False)

qna_input = {
    "question" :"Answer the following yes/no question. Can you write a whole Haiku in a single tweet?"
    }

qna_input_tokens = tokenizer(qna_input['question'], return_tensors="pt").input_ids
qna_op_tokens = model.generate(qna_input_tokens)
print(tokenizer.decode(qna_op_tokens[0],skip_special_tokens=True))


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


RuntimeError: ignored

# Adapter based method

Create a new class **CustomT5LayerNorm** similar to T5LayerNorm which will change input tensor to **CustomT5LayerNorm** hidden_size

In [55]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch


class CustomT5LayerNorm(torch.nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        Construct a layernorm module in the T5 style. No bias and no subtraction of mean.
        """
        super().__init__()
        self.weight = torch.nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):

        variance = hidden_states.to(torch.float32).pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)

        # convert into half-precision if necessary
        if self.weight.dtype in [torch.float16, torch.bfloat16]:
            hidden_states = hidden_states.to(self.weight.dtype)
        interim_layer = torch.nn.Linear(hidden_states.shape[2], self.weight.shape[0])
        hidden_states = interim_layer(hidden_states)

        return self.weight * hidden_states


In [56]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

model.decoder.final_layer_norm = CustomT5LayerNorm(256)
model.lm_head = torch.nn.Linear(in_features=256, out_features=32128, bias=False)

qna_input = {
    "question" :"Answer the following yes/no question. Can you write a whole Haiku in a single tweet?"
    }

qna_input_tokens = tokenizer(qna_input['question'], return_tensors="pt").input_ids
qna_op_tokens = model.generate(qna_input_tokens)
print(tokenizer.decode(qna_op_tokens[0],skip_special_tokens=True))

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


sleeping connection 23 neighbourhoodsynchronous revivalction employment forbidden GRAmur Father noihillFStörung bonus întâlnire Exist


# Train Squad dataset with question and context

In [1]:
from datasets import load_dataset
from transformers import DefaultDataCollator
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

In [2]:
squad = load_dataset("squad", split="train[:5000]")

In [3]:
squad = squad.train_test_split(test_size=0.2)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")


In [22]:
squad['train'][0]['question']

'What is the capacity of a solar hot water system?'

In [38]:
PREFIX = "Answer the following Question with context. "

def preprocess_squad(sample):
    questions = [PREFIX +  q for q in sample["question"]]
    inputs = tokenizer(
        questions,
        sample["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = sample["answers"]

    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


In [54]:
tokenized_squad = squad.map(preprocess_squad, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
data_collator = DefaultDataCollator()
model = AutoModelForQuestionAnswering.from_pretrained("google/flan-t5-small")

training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/content/my_awesome_qa_model is already a clone of https://huggingface.co/arunadiraju/my_awesome_qa_model. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.972726
2,4.679300,2.918360
3,4.679300,2.714592


TrainOutput(global_step=750, training_loss=4.290540120442708, metrics={'train_runtime': 680.0129, 'train_samples_per_second': 17.647, 'train_steps_per_second': 1.103, 'total_flos': 1218246137856000.0, 'train_loss': 4.290540120442708, 'epoch': 3.0})

In [67]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/Aug20_14-09-34_bf4b0a5360e3/events.out.tfevents.1692540579.bf4b0a5360e3.21211.0:   0%|       …

To https://huggingface.co/arunadiraju/my_awesome_qa_model
   6ede3cd..e9748b2  main -> main

   6ede3cd..e9748b2  main -> main

To https://huggingface.co/arunadiraju/my_awesome_qa_model
   e9748b2..7eb2d70  main -> main

   e9748b2..7eb2d70  main -> main



'https://huggingface.co/arunadiraju/my_awesome_qa_model/commit/e9748b29adb6c383e38ddb57efd72120a591ea43'

In [52]:
from evaluate import evaluator
task_evaluator = evaluator("question-answering")

data_test = load_dataset("squad", split="validation[:100]")
eval_results = task_evaluator.compute(
    model_or_pipeline="arunadiraju/my_awesome_qa_model",
    data=data_test,
    metric="squad",
    strategy="bootstrap",
    n_resamples=30
)

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [53]:
eval_results

{'exact_match': {'confidence_interval': (32.329825653079155,
   52.05398650484907),
  'standard_error': 5.409655055185308,
  'score': 44.0},
 'f1': {'confidence_interval': (48.35170704690194, 62.771536957046656),
  'standard_error': 4.05015825423107,
  'score': 57.81904761904761},
 'total_time_in_seconds': 5.793570883999564,
 'samples_per_second': 17.26051204036801,
 'latency_in_seconds': 0.05793570883999564}